In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya_speech.train.model.quartznet as quartznet
import malaya_speech
import tensorflow as tf
import numpy as np
import json






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [3]:
with open('malaya-speech-sst-vocab.json') as fopen:
    unique_vocab = json.load(fopen)

In [4]:
featurizer = malaya_speech.tf_featurization.STTFeaturizer(
    normalize_per_feature = True
)

In [5]:
X = tf.placeholder(tf.float32, [None, None])
X_len = tf.placeholder(tf.int32, [None])

In [6]:
batch_size = tf.shape(X)[0]
features = tf.TensorArray(dtype = tf.float32, size = batch_size, dynamic_size = True, infer_shape = False)
features_len = tf.TensorArray(dtype = tf.int32, size = batch_size)

init_state = (0, features, features_len)

def condition(i, features, features_len):
    return i < batch_size

def body(i, features, features_len):
    f = featurizer(X[i, :X_len[i]])
    f_len = tf.shape(f)[0]
    return i + 1, features.write(i, f), features_len.write(i, f_len)

_, features, features_len = tf.while_loop(condition, body, init_state)
features_len = features_len.stack()
padded_features = tf.TensorArray(dtype = tf.float32, size = batch_size)
maxlen = tf.reduce_max(features_len)

init_state = (0, padded_features)

def condition(i, padded_features):
    return i < batch_size

def body(i, padded_features):
    f = features.read(i)
    f = tf.pad(f, [[0, maxlen - tf.shape(f)[0]], [0,0]])
    return i + 1, padded_features.write(i, f)

_, padded_features = tf.while_loop(condition, body, init_state)
padded_features = padded_features.stack()
padded_features.set_shape((None, None, 80))

In [7]:
model = quartznet.Model(padded_features, features_len, mode = 'eval')
logits = tf.layers.dense(model.logits['outputs'], len(unique_vocab) + 1)
seq_lens = model.logits['src_length']
logits = tf.transpose(logits, [1, 0, 2])
logits = tf.identity(logits, name = 'logits')
seq_lens = tf.identity(seq_lens, name = 'seq_lens')


Instructions for updating:
Use `tf.keras.layers.SeparableConv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [8]:
# decoded = tf.nn.ctc_beam_search_decoder(logits, seq_lens, beam_width=100, top_paths=1, merge_repeated=True)
# preds = tf.sparse.to_dense(tf.to_int32(decoded[0][0]))
# preds = tf.identity(preds, 'logits')

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [10]:
var_lists = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, 'asr-quartznet-ctc-adam-part2/model.ckpt-270000')

INFO:tensorflow:Restoring parameters from asr-quartznet-ctc-adam-part2/model.ckpt-270000


In [11]:
y, _ = malaya_speech.load('husein-zolkepli.wav')
y1, _ = malaya_speech.load('khalil-nooh.wav')

In [12]:
padded, lens = malaya_speech.padding.sequence_1d([y, y1], return_len = True)

In [13]:
decoded = sess.run(logits, feed_dict = {X: padded, X_len: lens})

In [14]:
# results = []
# for i in range(len(decoded)):
#     results.append(malaya_speech.char.decode(decoded[i], lookup = unique_vocab).replace('<PAD>', ''))
# results

In [15]:
saver = tf.train.Saver()
saver.save(sess, 'asr-quartznet-ctc-output/model.ckpt')

'asr-quartznet-ctc-output/model.ckpt'

In [16]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 't_logits' in n.name
        or 'seq_lens' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'w2l_encoder/conv11/depthwise_kernel',
 'w2l_encoder/conv11/pointwise_kernel',
 'w2l_encoder/conv11/bn/gamma',
 'w2l_encoder/conv11/bn/moving_mean',
 'w2l_encoder/conv11/bn/moving_variance',
 'w2l_encoder/conv21/depthwise_kernel',
 'w2l_encoder/conv21/pointwise_kernel',
 'w2l_encoder/conv21/bn/gamma',
 'w2l_encoder/conv21/bn/moving_mean',
 'w2l_encoder/conv21/bn/moving_variance',
 'w2l_encoder/conv22/depthwise_kernel',
 'w2l_encoder/conv22/pointwise_kernel',
 'w2l_encoder/conv22/bn/gamma',
 'w2l_encoder/conv22/bn/moving_mean',
 'w2l_encoder/conv22/bn/moving_variance',
 'w2l_encoder/conv23/depthwise_kernel',
 'w2l_encoder/conv23/pointwise_kernel',
 'w2l_encoder/conv23/bn/gamma',
 'w2l_encoder/conv23/bn/moving_mean',
 'w2l_encoder/conv23/bn/moving_variance',
 'w2l_encoder/conv24/depthwise_kernel',
 'w2l_encoder/conv24/pointwise_kernel',
 'w2l_encoder/conv24/bn/gamma',
 'w2l_encoder/conv24/bn/moving_mean',
 'w2l_encoder/conv24/bn/moving_variance',
 'w2l_

In [17]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [18]:
freeze_graph('asr-quartznet-ctc-output', strings)

INFO:tensorflow:Restoring parameters from asr-quartznet-ctc-output/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 565 variables.
INFO:tensorflow:Converted 565 variables to const ops.
3549 ops in the final graph.


In [19]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph


In [21]:
g = load_graph('asr-quartznet-ctc-output/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
x_lens = g.get_tensor_by_name('import/Placeholder_1:0')
logits = g.get_tensor_by_name('import/logits:0')
seq_lens = g.get_tensor_by_name('import/seq_lens:0')
test_sess = tf.InteractiveSession(graph = g)
result = test_sess.run([logits, seq_lens], feed_dict = {x: padded, x_lens: lens})
result

[array([[[ -84.5379   ,  -82.79561  ,  -10.087287 , ...,  -51.145416 ,
          -149.3778   ,   16.79637  ],
         [ -65.14783  ,  -63.808704 ,  -10.393862 , ...,  -40.582966 ,
          -120.965164 ,   13.112476 ]],
 
        [[ -94.58125  ,  -94.09883  ,  -11.996426 , ...,  -62.57859  ,
          -150.48581  ,   16.297886 ],
         [ -77.958244 ,  -78.656425 ,   -9.595647 , ...,  -48.550175 ,
          -125.34512  ,   13.658209 ]],
 
        [[-126.261986 , -122.39516  ,  -10.872917 , ...,  -89.26167  ,
          -240.20914  ,   20.670303 ],
         [-103.20415  , -100.64028  ,   -8.713641 , ...,  -66.272194 ,
          -205.90721  ,   17.901423 ]],
 
        ...,
 
        [[-149.80501  , -145.71222  ,  -37.61645  , ...,  -94.94084  ,
          -271.87778  ,   24.769342 ],
         [ -16.619572 ,  -16.38736  ,   -1.8026912, ...,   -9.357312 ,
           -20.182959 ,    2.7737248]],
 
        [[-144.4081   , -141.93152  ,  -23.449245 , ..., -103.52729  ,
          -266.91586  

In [22]:
from tensorflow.tools.graph_transforms import TransformGraph

In [25]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'asr-quartznet-ctc-output/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['Placeholder', 'Placeholder_1'],
                                           ['logits', 'seq_lens'], transforms)
    
with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())